# Keep in Mind

**1. 사이트 페이지 몇 개를 살펴 보며 패턴을 찾는다.**

**2. 유연한 사고방식을 가진다. 웹사이트마다 레이아웃이 완전히 다르다.**

    - 각 웹사이트에서 어떤 정보를 찾을지,
    - 어떻게 찾을지 유연하게 생각하자. 
    
**3. 링크를 따라가다 보면 인터넷 어디든지 갈 수 있음을 유념하자. 항상 법, 규정, 보안 등에 주의하자.**

**4. 링크를 따라가는 크롤러를 만들기 전에, 다음과 같은 사항을 생각하자.**

    - 내가 수집하려는 데이터는 어떤 것인가?
        * 정해진 사이트 몇 개만 수집하면 되는가?
        * 내가 몰랐던 사이트에도 방문하는 크롤러가 필요한가?
        
    - 크롤러가 특정 웹사이트에 도달했을 때 어떻게 동작해야 할까?
        * 즉시 새 웹사이트 링크로 들어가야 하나?
        * 한 동안 현재 웹사이트에 머물면서 파고들고, 정보를 저장해야 하나?
        
    - 특정 사이트를 스크랩에서 제외할 필요가 있을까?(*ex. 비영어권 사이트, 성인용 사이트 등*)
    
    - 웹사이트가 내 방문을 알아차렸다면, 나 자신을 법적으로 보호할 수 있을까?(_18장_)
    
**5. 반드시, 실제 코드를 작성하기 전에 코드가 무슨 일을 하는지 다이어그램을 그리거나, 메모하는 습관을 갖는다. 크롤러가 복잡해질수록 시간이 절약되고, 좌절 경험이 적어질 것이다!**

# 크롤러 만들기

> 아래 단계로 갈수록, 크롤러의 동작이 많아진다.

**1.** 단일 도메인 내에서 페이지를 옮겨 다니며 각 링크를 방문하는 크롤러.

**2.** 단일 도메인 내에서, 페이지에 머무르는 동안 특정 정보를 수집하는 스크레이퍼를 추가한 크롤러.

**3.** 외부 링크를 따라 가면서, 여러 도메인 내에서 페이지에 대해 특정 정보를 수집하는 스크레이퍼를 추가한 크롤러.

## 1. 단일 도메인 내에서 페이지 이동
* 위키피디아에서 항목 페이지 간 이동하는 크롤러 제작
> BTS 페이지에서 TXT 페이지로 넘어가는 데 몇 번의 링크 이동이 필요할까?

In [9]:
# bts 위키피디아 페이지 읽어 오기
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError

def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        print("접속에 실패하였습니다. {}".format(e))
    try:
        bs = BeautifulSoup(html, 'html5lib')
        print("{} 페이지를 읽어 왔습니다.".format(bs.h1.get_text()))
    except AttributeError as e:
        print("error! no such tag!")
    return bs

bts_page = getTitle("https://en.wikipedia.org/wiki/BTS") # 성공
type(bts_page)

BTS 페이지를 읽어 왔습니다.


bs4.BeautifulSoup

In [14]:
# BTS 페이지에서 모든 링크 확인하기
for link in bts_page.findAll('a'):
    if 'href' in link.attrs: # a 태그에 href 속성이 있나 확인.
        print(link['href'])
        
# 결과 확인 : 원하지 않는 링크들도 있다.
# https://foundation.wikimedia.org/wiki/Privacy_policy
# https://wikimediafoundation.org/

/wiki/Wikipedia:Protection_policy#semi
#mw-head
#p-search
/wiki/BTS_(disambiguation)
/wiki/File:%E2%80%98LG_Q7_BTS_%EC%97%90%EB%94%94%EC%85%98%E2%80%99_%EC%98%88%EC%95%BD_%ED%8C%90%EB%A7%A4_%EC%8B%9C%EC%9E%91_(42773472410)_(cropped).jpg
/wiki/LG_Electronics
/wiki/V_(singer)
/wiki/J-Hope
/wiki/RM_(rapper)
/wiki/Kim_Seok-jin
/wiki/Jimin_(singer,_born_1995)
/wiki/Jungkook
/wiki/Suga_(rapper)
/wiki/Seoul
/wiki/K-pop
/wiki/Hip_hop_music
/wiki/Contemporary_R%26B
/wiki/Electronic_dance_music
/wiki/Big_Hit_Entertainment
/wiki/Pony_Canyon
/wiki/Def_Jam_Recordings
#cite_note-1
/wiki/Columbia_Records
#cite_note-2
http://bts.ibighit.com
/wiki/Kim_Seok-jin
/wiki/Suga_(rapper)
/wiki/J-Hope
/wiki/RM_(rapper)
/wiki/Jimin_(singer,_born_1995)
/wiki/V_(singer)
/wiki/Jungkook
/wiki/Hangul
https://en.wiktionary.org/wiki/%EB%B0%A9%ED%83%84
https://en.wiktionary.org/wiki/%EC%86%8C%EB%85%84
https://en.wiktionary.org/wiki/%EB%8B%A8
/wiki/Hanja
https://en.wiktionary.org/wiki/%E9%98%B2%E5%BD%88
https://en.wiktio

https://wikimediafoundation.org/
https://www.mediawiki.org/


In [21]:
# 정규표현식을 사용해 원하는 위키 항목 페이지들을 찾음.

# id가 bodyContent인 div 영역 하나 찾고,
# 정규표현식을 이용해 하이퍼링크 찾는다.

import re

for link in bts_page.find('div', {'id' : 'bodyContent'})\
                     .findAll('a', {'href' : re.compile("^(/wiki/)((?!:).)*$")}):
    if 'href' in link.attrs:
        print(link['href'])

/wiki/BTS_(disambiguation)
/wiki/LG_Electronics
/wiki/V_(singer)
/wiki/J-Hope
/wiki/RM_(rapper)
/wiki/Kim_Seok-jin
/wiki/Jimin_(singer,_born_1995)
/wiki/Jungkook
/wiki/Suga_(rapper)
/wiki/Seoul
/wiki/K-pop
/wiki/Hip_hop_music
/wiki/Contemporary_R%26B
/wiki/Electronic_dance_music
/wiki/Big_Hit_Entertainment
/wiki/Pony_Canyon
/wiki/Def_Jam_Recordings
/wiki/Columbia_Records
/wiki/Kim_Seok-jin
/wiki/Suga_(rapper)
/wiki/J-Hope
/wiki/RM_(rapper)
/wiki/Jimin_(singer,_born_1995)
/wiki/V_(singer)
/wiki/Jungkook
/wiki/Hangul
/wiki/Hanja
/wiki/Revised_Romanization_of_Korean
/wiki/McCune%E2%80%93Reischauer
/wiki/Kanji
/wiki/Hiragana
/wiki/Hepburn_romanization
/wiki/Kunrei-shiki_romanization
/wiki/Korean_language
/wiki/Revised_Romanization_of_Korean
/wiki/Boy_band
/wiki/Seoul
/wiki/Hip_hop_music
/wiki/Parallel_universes_in_fiction
/wiki/Big_Hit_Entertainment
/wiki/2_Cool_4_Skool
/wiki/Billboard_200
/wiki/The_Most_Beautiful_Moment_in_Life,_Part_2
/wiki/Wings_(BTS_album)
/wiki/Korean_Wave
/wiki/Recor

### 함수화_ver0
위의 과정을 함수화한다. 그러나, ver0에서는 계속 실행하면 재귀 호출 횟수 제한으로 인해 `RecursionError` 발생한다.



1. **`getLinks`** 함수
    * 페이지를 읽어 와서 그 페이지의 BeautifulSoup 객체를 반환.
    * 에러가 발생할 때, `None` 반환.
    * 읽어온 페이지에서 모든 항목 링크를 반환.
    

2. **메인 함수**
    * 시작 항목에서 getLinks를 호출해 항목 링크를 반환하고, 그 항목 링크를 무작위로 선택하여 getLinks를 다시 호출.
    * 프로그램이 끝나거나, 새 페이지에 항목 링크가 없을 때까지 위의 작업을 반복.


#### 작동 원리
    * 현재 시스템 시간으로 난수 시드 설정 : 프로그램을 실행할 때마다 새로운 무작위 경로 탐색.
    * "/wiki/"로 시작하는 URL을 받고, 그 앞에 위키백과 도메인 "http://en.wikipedia.org"을 붙인다. 이를 시작점으로 하여, 해당 위치의 HTML에서 BeautifulSoup 객체를 가져온다.
    * 위 페이지에서 모든 항목 링크 태그를 반환한다.
    * 메인 함수에서 루프를 실행하며 항목 링크를 무작위로 선택한다.
    * 새롭게 선택한 링크에서 `href` 속성을 추출한다.
    * 다시 URL을 받는 단계로 돌아가 페이지를 출력하고, URL에서 새 링크 목록을 가져온다. 

In [24]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import re
import datetime
import random

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    try:
        html = urlopen("http://en.wikipedia.org{}".format(articleUrl))
    except HTTPError as e:
        print("접속에 실패하였습니다. {}".format(e))
        return None
    try:
        bs = BeautifulSoup(html, 'html5lib')
        print("{} 페이지를 읽어 왔습니다.".format(bs.h1.get_text()))
    except AttributeError as e:
        print("error! no such tag!")
        return None
    return bs.find('div', {'id' : 'bodyContent'})\
              .findAll('a', {'href' : re.compile("^(/wiki/)((?!:).)*$")})


cnt = 0 # 링크 몇 번 탔는지 횟수 세기

links = getLinks('/wiki/BTS') # 시작점

while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    cnt += 1 # 새로운 기사 찾으면 일단 더하고
    if 'TXT_(band)' in newArticle: # 찾는 문서 나오면 멈추고
        break
    else: # 아니면 다시 진행
        links = getLinks(newArticle)

print(cnt)

BTS 페이지를 읽어 왔습니다.
/wiki/Shinee
Shinee 페이지를 읽어 왔습니다.
/wiki/Luhan_(entertainer)
Lu Han 페이지를 읽어 왔습니다.
/wiki/Exo-CBX
Exo-CBX 페이지를 읽어 왔습니다.
/wiki/Blooming_Days
Blooming Days 페이지를 읽어 왔습니다.
/wiki/Magic_(Exo-CBX_album)
Magic (Exo-CBX album) 페이지를 읽어 왔습니다.
/wiki/Ballad
Ballad 페이지를 읽어 왔습니다.
/wiki/Walter_Scott
Walter Scott 페이지를 읽어 왔습니다.
/wiki/Edvard_Grieg
Edvard Grieg 페이지를 읽어 왔습니다.
/wiki/Finland
Finland 페이지를 읽어 왔습니다.
/wiki/Mauritz_Stiller
Mauritz Stiller 페이지를 읽어 왔습니다.
/wiki/Helsinki
Helsinki 페이지를 읽어 왔습니다.
/wiki/Stockholm
Stockholm 페이지를 읽어 왔습니다.
/wiki/Leith
Leith 페이지를 읽어 왔습니다.
/wiki/Firth_of_Forth
Firth of Forth 페이지를 읽어 왔습니다.
/wiki/Charlestown,_Fife
Charlestown, Fife 페이지를 읽어 왔습니다.
/wiki/Gazetteer_for_Scotland
Gazetteer for Scotland 페이지를 읽어 왔습니다.
/wiki/Gazetteer
Gazetteer 페이지를 읽어 왔습니다.
/wiki/Composite_Gazetteer_of_Antarctica
Composite Gazetteer of Antarctica 페이지를 읽어 왔습니다.
/wiki/United_Kingdom
United Kingdom 페이지를 읽어 왔습니다.
/wiki/Government_of_Ireland
Government of Ireland 페이지를 읽어 왔습니다.
/wiki/Oireachta

URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

### 함수화_ver1

 보통 많은 웹사이트에서 내부 링크는 중복이다. 따라서 새로운 링크만 탐색하고, 그 안에서 다른 링크를 검색하도록 **set**을 사용한다.

1. 빈 URL넘겨 getLinks 호출.
2. getLinks 함수 내부에서 빈 URL과 위키피디아 도메인을 합쳐 위키피디아 첫 페이지로 간다.
3. 첫 페이지에서 각 항목 링크를 돌며 `pages`에 저장되어 있는지 검사.
4. 들어 있지 않은 링크라면 리스트에 출력하고, 화면에 출력한 다음, `getLinks` 함수 재귀적으로 호출.

In [27]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import HTTPError
import re
import datetime
import random

random.seed(datetime.datetime.now())

pages = set()
cnt = 0

def getLinks(pageUrl, targetWord):
    global pages
    global cnt
    
    try:
        html = urlopen("http://en.wikipedia.org{}".format(pageUrl))
    except HTTPError as e:
        print("접속에 실패하였습니다. {}".format(3))
        return None
    try:
        bs = BeautifulSoup(html, 'lxml')
        print("{} 페이지를 읽어 왔습니다.".format(bs.h1.get_text()))
    except AttributeError as e:
        print("{}! no such tag!".format(e))
        return None
    
    for link in bs.findAll('a', {'href':re.compile("^(/wiki/)((?!:).)*$")}):
        # 첫 페이지에서 시작하므로, bodyContent 조건 완화.
        
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages: # 이전에 발견하지 못한 페이지
                newPage = link.attrs['href']
                cnt += 1
                if cnt <= 30:
                    print(newPage)
                else:
                    print("...")
                    
                if targetWord in newPage: # 책 코드 수정
                    return cnt
                
                pages.add(newPage)
                getLinks(newPage, 'BTS')
        
getLinks('', 'BTS')

Main Page 페이지를 읽어 왔습니다.
/wiki/Wikipedia
Wikipedia 페이지를 읽어 왔습니다.
/wiki/English_Wikipedia
English Wikipedia 페이지를 읽어 왔습니다.
/wiki/Internet_encyclopedia
Online encyclopedia 페이지를 읽어 왔습니다.
/wiki/CD-ROM_encyclopedia
CD-ROM encyclopedia 페이지를 읽어 왔습니다.
/wiki/Encyclopedia
Encyclopedia 페이지를 읽어 왔습니다.
/wiki/Encyclopedia_(disambiguation)
Encyclopedia (disambiguation) 페이지를 읽어 왔습니다.
/wiki/Encyclopedia_(album)
Encyclopedia (album) 페이지를 읽어 왔습니다.
/wiki/Album
Album 페이지를 읽어 왔습니다.
/wiki/Album_(disambiguation)
Album (disambiguation) 페이지를 읽어 왔습니다.
/wiki/Album_%E2%80%93_Generic_Flipper
Album – Generic Flipper 페이지를 읽어 왔습니다.
/wiki/Flipper_(band)
Flipper (band) 페이지를 읽어 왔습니다.
/wiki/San_Francisco,_California
San Francisco 페이지를 읽어 왔습니다.
/wiki/San_Francisco_(disambiguation)
San Francisco (disambiguation) 페이지를 읽어 왔습니다.
/wiki/San_Francisco
San Francisco 페이지를 읽어 왔습니다.
/wiki/Consolidated_city-county
Consolidated city-county 페이지를 읽어 왔습니다.
/wiki/Political_divisions_of_the_United_States
Political divisions of the United State

Code of Canons of the Eastern Churches 페이지를 읽어 왔습니다.
...
Omnium in mentem 페이지를 읽어 왔습니다.
...
Magnum principium 페이지를 읽어 왔습니다.
...
Ad tuendam fidem 페이지를 읽어 왔습니다.
...
Ex Corde Ecclesiae 페이지를 읽어 왔습니다.
...
Indulgentiarum Doctrina 페이지를 읽어 왔습니다.
...
Pastor bonus 페이지를 읽어 왔습니다.
...
Roman Curia 페이지를 읽어 왔습니다.
...
Curia (Catholic Church) 페이지를 읽어 왔습니다.
...
Pontificalis Domus 페이지를 읽어 왔습니다.
...
Veritatis gaudium 페이지를 읽어 왔습니다.
...
Custom (canon law) 페이지를 읽어 왔습니다.
...
Matrimonial nullity trial reforms of Pope Francis 페이지를 읽어 왔습니다.
...
Second Vatican Council 페이지를 읽어 왔습니다.
...
Latin 페이지를 읽어 왔습니다.
...
Latin (disambiguation) 페이지를 읽어 왔습니다.
...
Latin 페이지를 읽어 왔습니다.
...
Colosseum 페이지를 읽어 왔습니다.
...
Colosseum (disambiguation) 페이지를 읽어 왔습니다.
...
Colosseum, Queensland 페이지를 읽어 왔습니다.
...
Queensland 페이지를 읽어 왔습니다.
...
Queensland (disambiguation) 페이지를 읽어 왔습니다.
...
Queensland, Calgary 페이지를 읽어 왔습니다.
...
Calgary 페이지를 읽어 왔습니다.
...
Calgary, Mull 페이지를 읽어 왔습니다.
...
Scottish Gaelic 페이지를 읽어 왔습니다.
...
Goidelic languages 페이지를 읽어 왔습

Catholic-Hierarchy.org 페이지를 읽어 왔습니다.
...
Alexa Internet 페이지를 읽어 왔습니다.
...
Amazon Alexa 페이지를 읽어 왔습니다.
...
Programmer 페이지를 읽어 왔습니다.
...
Program 페이지를 읽어 왔습니다.
...
Program management 페이지를 읽어 왔습니다.
...
Program Manager 페이지를 읽어 왔습니다.
...
Windows 3.1x 페이지를 읽어 왔습니다.
...
Windows NT 3.1 페이지를 읽어 왔습니다.
...
Windows NT 페이지를 읽어 왔습니다.
...
Microsoft 페이지를 읽어 왔습니다.
...
Microsoft Redmond campus 페이지를 읽어 왔습니다.
...
Redmond, Washington 페이지를 읽어 왔습니다.
...
Redmond, Western Australia 페이지를 읽어 왔습니다.
...
Western Australia 페이지를 읽어 왔습니다.
...
Flag of Western Australia 페이지를 읽어 왔습니다.
...
Glossary of vexillology 페이지를 읽어 왔습니다.
...
Nomenclature 페이지를 읽어 왔습니다.
...
Nomenklatura 페이지를 읽어 왔습니다.
...
Soviet Union 페이지를 읽어 왔습니다.
...
USSR (disambiguation) 페이지를 읽어 왔습니다.
...
Soviet Union 페이지를 읽어 왔습니다.
...
CCCP (disambiguation) 페이지를 읽어 왔습니다.
...
Soviet Union 페이지를 읽어 왔습니다.
...
Soviet (disambiguation) 페이지를 읽어 왔습니다.
...
Romanization of Russian 페이지를 읽어 왔습니다.
...
Transliteration 페이지를 읽어 왔습니다.
...
Translation 페이지를 읽어 왔습니다.
...
Translation (disa

Holstein 페이지를 읽어 왔습니다.
...
Holstein (disambiguation) 페이지를 읽어 왔습니다.
...
Schleswig-Holstein 페이지를 읽어 왔습니다.
...
Province of Schleswig-Holstein 페이지를 읽어 왔습니다.
...
Prussia 페이지를 읽어 왔습니다.
...
Prussia (disambiguation) 페이지를 읽어 왔습니다.
...
Preußen 페이지를 읽어 왔습니다.
...
Preussen (ship) 페이지를 읽어 왔습니다.
...
German Empire 페이지를 읽어 왔습니다.
...
German Empire (disambiguation) 페이지를 읽어 왔습니다.
...
Holy Roman Empire 페이지를 읽어 왔습니다.
...
Roman Empire 페이지를 읽어 왔습니다.
...
Roman Empire (disambiguation) 페이지를 읽어 왔습니다.
...
Autocracy 페이지를 읽어 왔습니다.
...
Dominion 페이지를 읽어 왔습니다.
...
Dominion (disambiguation) 페이지를 읽어 왔습니다.
...
Dominion, Nova Scotia 페이지를 읽어 왔습니다.
...
List of sovereign states 페이지를 읽어 왔습니다.
...
Dependent territory 페이지를 읽어 왔습니다.
...
Territory 페이지를 읽어 왔습니다.
...
Territory (disambiguation) 페이지를 읽어 왔습니다.
...
Box office territory 페이지를 읽어 왔습니다.
...
Film industry 페이지를 읽어 왔습니다.
...
Film crew 페이지를 읽어 왔습니다.
...
The Film Crew 페이지를 읽어 왔습니다.
...
Walk the Angry Beach 페이지를 읽어 왔습니다.
...
Five Minutes to Love 페이지를 읽어 왔습니다.
...
John Hayes (dire

List of states in the Holy Roman Empire (Q) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (R) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (S) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (T) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (U) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (V) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (W) 페이지를 읽어 왔습니다.
...
List of states in the Holy Roman Empire (Z) 페이지를 읽어 왔습니다.
...
List of free imperial cities 페이지를 읽어 왔습니다.
...
Free imperial city 페이지를 읽어 왔습니다.
...
Imperial City 페이지를 읽어 왔습니다.
...
Free imperial city 페이지를 읽어 왔습니다.
...
Reichstadt 페이지를 읽어 왔습니다.
...
Reichstadt Agreement 페이지를 읽어 왔습니다.
...
Zákupy 페이지를 읽어 왔습니다.
...
Regions of the Czech Republic 페이지를 읽어 왔습니다.
...
Czech lands 페이지를 읽어 왔습니다.
...
Kraj 페이지를 읽어 왔습니다.
...
Krai 페이지를 읽어 왔습니다.
...
KRAI 페이지를 읽어 왔습니다.
...
KRAI (AM) 페이지를 읽어 왔습니다.
...
City of license 페이지를 읽어 왔습니다.
...
Philippines 페이지를 읽어 왔습니다.
...
Philippine, Netherl

KeyboardInterrupt: 

## 2. 이동하며 특정 정보를 스크레이핑하는 크롤러
* 페이지 제목, 첫 번째 문단, 편집 페이지를 가리키는 링크(존재한다면)를 스크레이핑하며, 이동하는 크롤러를 제작한다.
* 아직까지 수집한 데이터를 저장하지는 않고 출력만 한다. 저장하는 것은 *6장*에서 다루도록 한다.
---

1. 페이지 패턴 탐색
    * 제목의 위치 : h1 태그 안.
    * (교재와 다름) 첫 문단 : `div#mf-section-0`
    * (교재와 다름) (if any) 편집 링크 : `a#ca-edit -> href`


2. 1의 ver1 기본 크롤링 코드를 수정해, 크롤링과 데이터 스크레이핑(출력 가능) 기능이 있는 프로그램 제작. -> 웹 페이지 태그 언제나 변경될 수 있으므로, 에러 확인하는 코드 필요하다.

3. 함수 설명
    * 출력되는 컨텐츠를 명확히 구분하기 위해 대시 추가.
    * 원하는 정보 모두가 페이지에 있다고 확신할 수 없으므로, 있을 확률이 높은 순서대로 정렬해서 출력.
    
#### 편집 페이지 밑에서는 되는데 왜 여기서는 안 될까?

In [82]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError
import re
import random
import datetime

pages = set()

random.seed(datetime.datetime.now())

def getLinks(pageUrl, targetWord):
    global pages
    
    try:
        html = urlopen("http://en.wikipedia.org"+pageUrl) # 1의 함수 수정.
    except HTTPError as e:
        print("접속에 실패하였습니다. {}".format(e))
        return None
    try:
        bs = BeautifulSoup(html, 'lxml')
        print("제목 : {}".format(bs.h1.get_text())) # 제목
        print("첫 문단 : {}".format(bs.find('div', {'class' : 'mw-parser-output'})\
                                      .findAll('p')[1].get_text())) # 첫 문단
        print("편집 페이지 : {}".format(bs.find('a', {'id' : "ca-edit"}).attrs['href'])) # 편집 페이지
    except AttributeError as e:
        print("없는 정보가 있습니다. No Worries Though!")
    except IndexError as e:
        print("페이지 구조가 다릅니다. No Worries Though!")    
    for link in bs.findAll('a', href = re.compile("^(/wiki/)")):
        # 제약 조건 완화. 항목 페이지 아니어도 되게 했다.
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print("-------------\n"+"새로 찾은 페이지:"+newPage)           
                pages.add(newPage)
                getLinks(newPage, targetWord)
            
getLinks('/wiki/Jungkook', 'BTS')

제목 : Jungkook
첫 문단 : Jeon Jung-kook (Korean: 전정국; born September 1, 1997), better known mononymously as Jungkook (stylized as Jung Kook), is a South Korean singer, songwriter and record producer. He is a member and vocalist of the South Korean boy band BTS.[2]

없는 정보가 있습니다. No Worries Though!
-------------
새로 찾은 페이지:/wiki/Wikipedia:Protection_policy#semi
제목 : Wikipedia:Protection policy
첫 문단 : 

없는 정보가 있습니다. No Worries Though!
-------------
새로 찾은 페이지:/wiki/Wikipedia:Requests_for_page_protection
제목 : Wikipedia:Requests for page protection
첫 문단 : After a page has been protected, the protection is listed in the page history and logs with a short description indicating why it was protected, and the article is listed on Special:Protectedpages. Further discussion should take place on the Talk page of the article. In the case of full protection due to edit warring, admins do not revert to specific versions of the page, except to get rid of obvious vandalism.

없는 정보가 있습니다. No Worries Though!
-

KeyboardInterrupt: 

In [74]:
html = urlopen("https://en.m.wikipedia.org/wiki/TXT_(band)")
bs = BeautifulSoup(html, 'html.parser')
print(bs.find('a', {'id' : "ca-edit"})['href'])

/w/index.php?title=TXT_(band)&action=edit&section=0


## 3. 여러 도메인을 이동하며 특정 정보를 스크레이핑하는 크롤러

1. 페이지에서 발견된 내부 링크를 모두 목록으로 만듦.

2. 페이지에서 발견된 외부 링크를 모두 목록으로 만듦.
    * 웹사이트의 첫 페이지에 항상 외부 링크가 있다는 보장이 없다.
    * 따라서 외부 링크를 찾을 때까지 웹사이트를 재귀적으로 파고든다.
    * 실무에서 이 코드를 실행한다면, 재귀 제한에 걸리게 되므로 **반드시** 예외를 처리하는 코드를 넣어야 한다.

3. 무작위로 외부 링크를 타고 이동.

4. 외부 링크를 검색하고, 각 링크마다 메모를 남김.
    * 하나의 동작에서는 내부 링크를 수집한다.
    * 또 다른 동작에서는 외부 링크를 수집한다.
    * 두 동작이 연관되어 수행된다.

In [97]:
from urllib.request import urlopen
from urllib.parse import urlparse
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
allExtLinks = set()
allIntLinks = set()
random.seed(datetime.datetime.now())

# 페이지에서 발견된 내부 링크 모두 목록으로 만든다.
def getInternalLinks(bs, includeUrl): # includeUrl : 현재 페이지 url
    includeUrl = "{}://{}".format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks = []
    # 현재 페이지에서 /로 시작하는 링크 찾기
    for link in bs.findAll('a', href = re.compile("^(/|.*" + includeUrl + ")")): # 콤마, 괄호는 그냥 중간에 변수 넣을라고 해준 것.
        print(link) # 질문 해결을 위한 확인 용도
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if link.attrs['href'].startswith('/'):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(links.attrs['href'])
    return internalLinks

def getExternalLinks(bs, excludeUrl):
    externalLinks = []
    # 현재 url을 포함하지 않으면서, http나 www로 시작하는 링크 찾기 : 외부 링크라는 의미이므로.
    for link in bs.findAll('a',
                          href = re.compile("^(http|www)((?!" + excludeUrl + ").)*$")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    try:
        html = urlopen(startingPage)
    except HTTPError as e:
        print("접속되지 않았습니다. 에러 내용 : {}".format(e))
    try:
        bs = BeautifulSoup(html, 'lxml')
        externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc) # 외부 링크 뽑아 온다.
    except AttributeError as e:
        print("Error! No such Tag!")
    else:
        if len(externalLinks) == 0:
            print("외부 링크가 없습니다. 다른 링크 찾기를 시도합니다.")
            domain = "{}://{}".format(urlparse(startingPage).scheme, urlparse(startingPage).netloc)
            internalLinks = getInternalLinks(bs, domain) # 내부 링크 랜덤으로 찾고, 거기서 randomExternal 함수 이용해서 랜덤으로 외부 링크로 간다.
            return getRandomExternalLink(internalLinks[random.randint(0, len(internalLinks))])
        else:
            return externalLinks[random.randint(0, len(externalLinks)-1)] # 외부링크 있으면 랜덤으로 가면 됨.

def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print("랜덤으로 찾은 외부 링크 : {}".format(externalLink))
    followExternalOnly(externalLink)

followExternalOnly("http://oreilly.com")

랜덤으로 찾은 외부 링크 : https://oreilly.formulated.by/scme-miami-2020/
랜덤으로 찾은 외부 링크 : https://www.linkedin.com/in/julianicolewatson/


HTTPError: HTTP Error 999: Request denied

In [90]:
followExternalOnly("https://www.naver.com")

랜덤으로 찾은 외부 링크 : http://newsstand.naver.com/?list=ct8&pcode=387
랜덤으로 찾은 외부 링크 : http://www.kwnews.co.kr/
랜덤으로 찾은 외부 링크 : http://www.jjan.kr/
랜덤으로 찾은 외부 링크 : https://www.facebook.com/jeonbukilbo/
랜덤으로 찾은 외부 링크 : http://l.facebook.com/l.php?u=http%3A%2F%2Fwww.jjan.kr%2Fnews%2FarticleView.html%3Fidxno%3D2079382&h=AT19vMabQ8F1V3AmV8g7PCgwrNFadjUn9nmK4Ndko4buoc3Pv9OB6MB_yFSFBgjtpjbC4xHUnVUG81d4yIMR2EmiTjSRKnjK_0WLhWa278sC9uN92HWPsEAVJOnhonJzNecTdbCimV-b6fkHzaUaKGTzD9oAUvnt4fdaVryP4jvs
외부 링크가 없습니다. 다른 링크 찾기를 시도합니다.


ValueError: empty range for randrange() (0,0, 0)